In [1]:
# Création du dossier WebCrawler contenant l'ensemble des fichiers utiles au fonctionnement de scrapy
!scrapy startproject MangaCrawler

Error: scrapy.cfg already exists in /home/yannis/Projects/Ecole_IPSSI/2022-2023/WebScraping/Scrapy/WebScraping/MangaCrawler


In [9]:
# ***************** création du projet Manga *********

!cd MangaCrawler && scrapy genspider mangascantrad https://www.japscan.me/

Created spider 'mangascantrad' using template 'basic' in module:
  MangaCrawler.spiders.mangascantrad


In [ ]:
# ***************** Ajout au fichier items.py & Liste des données qui seront récupérés par manga *********

class MangacrawlerItem(scrapy.Item):
    # define the fields for your item here like:
    title = scrapy.Field()
    img = scrapy.Field()
    rating = scrapy.Field()
    last_chapter = scrapy.Field()
    link = scrapy.Field()
    genres = scrapy.Field()
    published_date = scrapy.Field()
    state = scrapy.Field()
    nb_comments = scrapy.Field()
    pass

In [ ]:
# ***************** Liste des commandes********************

#1. Le titre du manga
response.css('div#main div.card-body h1::text').get()

#2. Le lien de l'image du manga
response.css('div#main div.card-body img').attrib['src']

#3. L'origine du manga
response.css('div#main div.card-body p.mb-2')[1].css('span::text')[1].get()

#4. La description du manga
response.css('div#main div.card-body p.list-group-item.list-group-item-primary.text-justify::text').get()

#5. Le dernier chapitre du manga
response.css('div#chapters_list div.collapse.show div.chapters_list.text-truncate').get().split(':')[0].split('\t')[-1]

#5. Le lien de la page détail du manga
response.request.url

#8. Les genres du manga
response.css('div#main div.card-body p.mb-2')[5].get().split('</span')[1].split('\t')[7].strip()

#6. Le type du manga
response.css('div#main div.card-body p.mb-2')[4].get().split('</span')[1].split('\t')[7].strip()

#6. La date de publication du manga
response.css('div#main div.card-body p.mb-2')[3].get().split('</span')[1].split('\t')[7].strip()

#7. L'état du manga
response.css('div#main div.card-body p.mb-2')[2].get().split('</span')[1].split('\t')[6].strip()


In [14]:
# La class MangascantradSpider
import scrapy
from scrapy import Request
from MangaCrawler.items import MangaGenres, MangacrawlerItem, DataBase
import sqlalchemy as db
from sqlalchemy.orm import declarative_base, relationship

class MangascantradSpider(scrapy.Spider):
    name = 'mangascantrad'
    allowed_domains = ['manga-scantrad.net']

    # Liste des urls par pages
    start_urls = [f'http://manga-scantrad.net/manga/page/{n}' for n in range(1, 31)]

    # Création de la base de données
    database = DataBase('database_manga')

    # Creation des tables avec une relation ManyToMany
    Base = declarative_base()
    association_table = db.Table(
        "mangas_assoc_genres",
        Base.metadata,
        db.Column("mangas_id", db.ForeignKey("mangas.id_")),
        db.Column("mangas_genres_id", db.ForeignKey("mangas_genres.id_")),
    )
    database.create_table('mangas_genres',
                          id_=db.Integer,
                          name=db.String,
                          )

    database.create_table('mangas',
                          id_=db.Integer,
                          title=db.String,
                          img=db.String,
                          rating=db.String,
                          last_chapter=db.String,
                          link=db.String,
                          genres=db.String,
                          published_date=db.String,
                          state=db.String,
                          nb_comments=db.String,
                          children=relationship("mangas_genres", secondary=association_table)
                          )

    def start_requests(self):
        print("iok")
        # for url in self.start_urls:
        #     yield Request(url=url, callback=self.parse_manga)

    def parse_manga(self, response):
        mangas = response.css('div.js-categories-seasonal.js-block-list.list table tr')[1:]
        for manga in mangas:
            item = MangacrawlerItem()

            # Nom manga
            try:
                item['name'] = manga.css('td')[1].css('a.hoverinfo_trigger.fw-b strong::text').get()
            except:
                item['name'] = 'None'

            # Image manga
            try:
                item['img'] = manga.css('td')[0].css('a.hoverinfo_trigger img').attrib['data-src']
            except:
                item['img'] = 'None'

            # Description manga
            try:
                item['description'] = manga.css('td')[1].css('div.pt4::text').get()
            except:
                item['description'] = 'None'

            # Ajouter dans la base de données
            self.database.add_row('manga',
                                  name=item['name'],
                                  img=item['img'],
                                  description=item['description']
                                  )
            yield item



ModuleNotFoundError: No module named 'MangaCrawler.items'

In [ ]:
# Exécution de scrapy
!cd MangaCrawler/MangaCrawler/spiders && scrapy crawl mangascantrad -o mangascantrad.csv

*********Table : 'mangas_types' are created succesfully************
*********Table : 'mangas' are created succesfully************
*********Table Relationship : 'mangas_assoc_types' are created succesfully************
2022-10-20 17:13:38 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: MangaCrawler)
2022-10-20 17:13:38 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.9.2 (default, Feb 28 2021, 17:03:44) - [GCC 10.2.1 20210110], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.0-19-amd64-x86_64-with-glibc2.31
2022-10-20 17:13:38 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'MangaCrawler',
 'NEWSPIDER_MODULE': 'MangaCrawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['MangaCrawler.spiders']}
2022-10-20 17:13:38 [py.warnings] WARNING: /home/yannis/.local/lib/python3.9/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is

2022-10-20 17:13:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/07-ghost/>
{'description': 'Il y a de cela plus de mille ans, le monde était divisé en '
                'deux puissants pays de force égale : l’empire de Barsburg, '
                'sous la protection divine de l’Œil du dieu Raphaël, et le '
                'royaume de Raggs, protégé par l’Œil du dieu Michael. Lorsque '
                'Raggs fut anéanti par la puissance prodigieuse de Barsburg, '
                'le destin du monde bascula, en même temps que celui de Teito '
                'Klein, un jeune garçon orphelin qui, mystérieusement, n’a '
                'plus aucun souvenir de son passé...',
 'genres': 'Josei',
 'img': '/imgs/mangas/07-ghost.jpg',
 'last_chapter': 'https://www.japscan.me/manga/07-ghost/',
 'link': 'https://www.japscan.me/manga/07-ghost/',
 'origin': 'Japon',
 'published_date': '',
 'state': '',
 'title': 'Manga 07 Ghost',
 'type': '2005'}
2022-10-20 17:13:39 

2022-10-20 17:13:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/100-jours-avant-ta-mort/>
{'description': 'Taro a enfin pu avouer à son amie d’enfance Umi qu’il l’aime '
                'depuis toujours ! Mais à cet instant, s’affiche le nombre 100 '
                'devant la jeune fille… Car Taro a un pouvoir spécial : celui '
                'de visualiser un compte à rebours sur tous les êtres vivants '
                'qui ont moins de 100 jours à vivre ! Ensemble, le couple va '
                'faire face au destin et tenter d’arrêter le décompte '
                'fatidique…',
 'genres': 'Shōjo',
 'img': '/imgs/mangas/100-jours-avant-ta-mort.jpg',
 'last_chapter': 'https://www.japscan.me/manga/100-jours-avant-ta-mort/',
 'link': 'https://www.japscan.me/manga/100-jours-avant-ta-mort/',
 'origin': 'Japon',
 'published_date': '',
 'state': '',
 'title': 'Manga 100 Jours Avant Ta Mort',
 'type': '2018'}
2022-10-20 17:13:39 [scrapy.core.scraper] DEBUG

2022-10-20 17:13:40 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/1-3-romantica/>
{'description': 'Après des semaines à regarder le garçon le plus populaire de '
                "l'école, Hana parvient enfin à lui parler, et même à attirer "
                'un peu son attention. Un jour, elle le surprend à verser '
                "quelques larmes et c'est alors qu'elle lui avoue "
                "maladroitement ses sentiments, alors qu'elle ne prévoyait que "
                'de le réconforter...',
 'genres': 'Shōjo',
 'img': '/imgs/mangas/1-3-romantica.jpg',
 'last_chapter': 'https://www.japscan.me/manga/1-3-romantica/',
 'link': 'https://www.japscan.me/manga/1-3-romantica/',
 'origin': 'Japon',
 'published_date': '',
 'state': '',
 'title': 'Manga 1-3 Romantica',
 'type': '2008'}
2022-10-20 17:13:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/1hp-club/> (referer: None)
2022-10-20 17:13:40 [scrapy.core.engine] DEBUG: Cra

2022-10-20 17:13:40 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/1-second/>
{'description': 'Un pompier légendaire avec un taux de sauvetage réussi de '
                '100%. Quelle est sa capacité spéciale ?',
 'genres': 'Drame, Surnaturel, Action, Aventure, Webtoon',
 'img': '/imgs/mangas/1-second.jpg',
 'last_chapter': 'https://www.japscan.me/manga/1-second/',
 'link': 'https://www.japscan.me/manga/1-second/',
 'origin': 'Corée Du Sud',
 'published_date': '',
 'state': '',
 'title': 'Manhwa 1 Second',
 'type': '2019'}
2022-10-20 17:13:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/a-girl-like-alien/> (referer: None)
2022-10-20 17:13:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/agharta/> (referer: None)
2022-10-20 17:13:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/a-girls/> (referer: None)
2022-10-20 17:13:40 [scrapy.core.scraper] ERROR: Spider error pro

...

2022-10-20 17:13:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/annoying-alice/> (referer: None)
2022-10-20 17:13:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/annarasumanara/>
{'description': "Croyez vous en la magie ? L'aventure de notre héroïne "
                'commence par cette question.',
 'genres': 'Drame, Mystère, School Life, Surnaturel, Tranche De Vie, Romance, '
           'Psychologie',
 'img': '/imgs/mangas/annarasumanara.jpg',
 'last_chapter': 'https://www.japscan.me/manga/annarasumanara/',
 'link': 'https://www.japscan.me/manga/annarasumanara/',
 'origin': 'Corée Du Sud',
 'published_date': '2010',
 'state': 'Terminé',
 'title': 'Manhwa Annarasumanara',
 'type': 'Shōjo'}
2022-10-20 17:13:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/all-in/> (referer: None)
2022-10-20 17:13:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/anne-freaks/> (refere

2022-10-20 17:13:47 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.japscan.me/manga/all-new-amazing-spider-man/> (referer: None)
Traceback (most recent call last):
  File "/home/yannis/Projects/Ecole_IPSSI/2022-2023/WebScraping/Scrapy/WebScraping/MangaCrawler/MangaCrawler/spiders/mangascantrad.py", line 80, in parse_manga
    item['origin'] = manga.css('div#main div.card-body p.mb-2')[2].css('span::text')[1].get()
  File "/home/yannis/.local/lib/python3.9/site-packages/parsel/selector.py", line 70, in __getitem__
    o = super(SelectorList, self).__getitem__(pos)
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yannis/.local/lib/python3.9/site-packages/scrapy/utils/defer.py", line 240, in iter_errback
    yield next(it)
  File "/home/yannis/.local/lib/python3.9/site-packages/scrapy/utils/python.py", line 338, in __next__
    return next(self.data)
  File "/ho

2022-10-20 17:13:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/all-you-need-is-kill/>
{'description': "Dans un futur proche, des hordes d'extraterrestres "
                'extrêmement bien organisées, appelées les Mimics, ont livré '
                'une bataille acharnée contre la Terre, réduisant les grandes '
                "villes en cendres et causant la mort de millions d'êtres "
                "humains. Aucune armée au monde n'est à même de rivaliser avec "
                'la rapidité, la violence et les capacités exceptionnelles des '
                'combattants Mimics, très bien armés, ou de leurs chefs dotés '
                'de pouvoirs télékinésiques. Mais à présent, les armées du '
                'monde ont réuni leurs forces pour une ultime offensive contre '
                'les extraterrestres…',
 'genres': 'Seinen',
 'img': '/imgs/mangas/all-you-need-is-kill.jpg',
 'last_chapter': 'https://www.japscan.me/manga/all-you-need-is-kil

2022-10-20 17:13:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/amachin-wa-jishou/> (referer: None)
2022-10-20 17:13:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/amagami-love-goes-on-morishima-haruka-hen/> (referer: None)
2022-10-20 17:13:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/alpha/>
{'description': "L'agent dénué de sentiment, Alpha, est chargé d'exécuter une "
                'mission secrète...',
 'genres': 'Seinen',
 'img': '/imgs/mangas/alpha.jpg',
 'last_chapter': 'https://www.japscan.me/manga/alpha/',
 'link': 'https://www.japscan.me/manga/alpha/',
 'origin': 'Corée Du Sud',
 'published_date': '',
 'state': '',
 'title': 'Manhwa Alpha',
 'type': '2019'}
2022-10-20 17:13:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/amaama-to-inazuma/> (referer: None)
2022-10-20 17:13:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/ma

2022-10-20 17:13:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/amagami-san-chi-no-enmusubi/>
{'description': 'Kamihate Uryuu est orphelin depuis son enfance. Un jour, il '
                "reçoit une invitation l'invitant à séjourner dans le temple "
                "Shinto local. En effet, tout ce qu'il veut, c'est de pouvoir "
                "étudier afin de réaliser son rêve d'entrer dans une des "
                'meilleures écoles de médecine du pays. Cependant, dès son '
                'arrivée au temple, il fait la rencontre de trois belles '
                'soeurs qui sont les prêtresses du temple. Le prêtre en chef '
                'fait alors une proposition à Uryuu : épouser une des soeurs '
                'et reprendre la succession du temple ! Commence alors un '
                "conte rempli d'amours et de miracles !",
 'genres': 'Shōnen',
 'img': '/imgs/mangas/amagami-san-chi-no-enmusubi.jpg',
 'last_chapter': 'https://www.japscan.me

In [ ]:
import pandas as pd

df = pd.read_csv('MangaCrawler/MangaCrawler/spiders/mangascantrad.csv')
df